## Часть 3.
---
В данной части я преобразую полученные на предыдущем этапе данные и приведу их к нормальному виду для дальнейшей аналитики

In [16]:
import re
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [17]:
df= pd.read_csv("df_completed.csv", sep=',',encoding='utf-8-sig')

df.isnull().sum()
df.info()

df['id'] = df.index
df['name'] = df['name'].str.strip()
df['district'] = df['district'].str.strip()

df['latitude'] = pd.to_numeric(df['latitude'].str.replace('\"',''))
df['longitude'] = pd.to_numeric(df['longitude'].str.replace('\"',''))

df['bld_age'] = 2019-df['year_built']

df['proj_area'] = pd.to_numeric(df['proj_area'].str.replace('\,',''))

print(df[~df['units'].str.isnumeric()]['units'])
df['units'] = pd.to_numeric(df['units'], errors = 'coerce')
missing_idx = np.isnan(df['units'])
missing_unit_df = df[missing_idx]
plt.hist(df['units'][~missing_idx])
plt.show()
print(df['units'][~missing_idx].describe())
df['units'] = df.groupby("district")["units"].transform(lambda x: x.fillna(x.median()))

def find_dist(input_str):
    input_str = str(input_str)
    dist = re.findall(r"[-+]?\d*\.\d+|[-+]?\d+", input_str)[0]
    unit = re.findall(r" km ", input_str)
    if (len(unit)!=0): dist_km = float(dist)
    else: dist_km = float(dist)/1000
    return(dist_km)
    
df['shops']=df['shops'].str.replace('\'','').str.split('\,')
df['shops'][0]
col_list = ['dist_shop_'+str(i) for i in range(1, 6)]
df[col_list]= df['shops'].apply(pd.Series)
for col in col_list: df[col]=df[col].apply(lambda x: find_dist(x))

df['schools']=df['schools'].str.replace('\"','\'').str.split('\', \'')
df['schools'][0]
len_school = df['schools'].apply(lambda x: len(x))
col_list = ['dist_school_'+str(i) for i in range(1, 6)]
df[col_list]= df['schools'].apply(pd.Series)
for col in col_list: df[col]=df[col].apply(lambda x: find_dist(x))

df['restaurants']=df['restaurants'].str.replace('\"','\'').str.split('\', \'')
df['restaurants'][0]
col_list = ['dist_food_'+str(i) for i in range(1, 6)]
df[col_list]= df['restaurants'].apply(pd.Series)
for col in col_list: df[col]=df[col].apply(lambda x: find_dist(x))

df['hospital'][0]
df['hospital']=df['hospital'].apply(lambda x: find_dist(x))

col_list = ['Elevator','Parking','Security','CCTV','Pool','Sauna','Gym',\
            'Garden','Playground','Shop','Restaurant','Wifi']
df['amenities']=df['amenities'].str.replace('\'','')
len(df['amenities'][0])
df['amenities'][0]
df[col_list] = df['amenities'].str.split(",",expand=True,)
df[col_list] = df[col_list].apply(lambda x: x.str.strip())
df[col_list] = df[col_list].apply(lambda x: x.str.replace('\[','')).\
apply(lambda x: x.str.replace('\]',''))
df[col_list] = df[col_list].apply(pd.to_numeric)

plt.hist(df['price_sqm'])
plt.show()
print(df['price_sqm'].describe())

df['transportation']=df['transportation'].str.replace('\'','').str.split('\,')
len_chk = df['transportation'].apply(lambda x: len(x))
df['transportation'][0]

col_list = ['tran_type'+str(i) for i in range(1, 6)]
df[col_list]= df['transportation'].apply(pd.Series).iloc[:,[0,3,6,9,12]]
for col in col_list: df[col]=df[col].apply(lambda x: re.findall("(expressway|mrt|bts)", x)[0])
col_list = ['tran_name'+str(i) for i in range(1, 6)]
df[col_list]= df['transportation'].apply(pd.Series).iloc[:,[1,4,7,10,13]].apply(lambda x: x.str.strip())
col_list = ['dist_tran_'+str(i) for i in range(1, 6)]
df[col_list]= df['transportation'].apply(pd.Series).iloc[:,[2,5,8,11,14]]
for col in col_list: df[col]=df[col].apply(lambda x: find_dist(x))

df.drop(['shops', 'schools', 'restaurants', 'amenities',
         'transportation','change_last_q', 'change_last_y', 'rental_yield',
         'change_last_y_rental_price', 'price_hist'], axis=1, inplace=True)

df.isnull().sum()
df.info()

df.to_csv(r"df_cleaned.csv", index=False, encoding='utf-8-sig')

FileNotFoundError: [Errno 2] No such file or directory: 'df_completed.csv'